预处理

In [5]:
from gensim import models
from gensim import corpora

In [31]:
texts = [['human', 'interface', 'computer'],
['survey', 'user', 'computer', 'system', 'response', 'time'],
['eps', 'user', 'interface', 'system'],
['system', 'human', 'system', 'eps'],
['user', 'response', 'time'],
['trees'],
['graph', 'trees'],
['graph', 'minors', 'trees'],
['graph', 'minors', 'survey']]

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
print(corpus)
print(dictionary.token2id)

[[(0, 1), (1, 1), (2, 1)], [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)], [(2, 1), (5, 1), (7, 1), (8, 1)], [(1, 1), (5, 2), (8, 1)], [(3, 1), (6, 1), (7, 1)], [(9, 1)], [(9, 1), (10, 1)], [(9, 1), (10, 1), (11, 1)], [(4, 1), (10, 1), (11, 1)]]
{'computer': 0, 'human': 1, 'interface': 2, 'response': 3, 'survey': 4, 'system': 5, 'time': 6, 'user': 7, 'eps': 8, 'trees': 9, 'graph': 10, 'minors': 11}


In [36]:
dictionary.items()

ItemsView(<gensim.corpora.dictionary.Dictionary object at 0x000001D45A10EB70>)

训练TFIDF

In [8]:
tfidf=models.TfidfModel(corpus)

In [9]:
print(tfidf)

TfidfModel(num_docs=9, num_nnz=28)


In [11]:
doc_dow=[(0,1),(3,1)]
tfidf[doc_dow]

[(0, 0.70710678118654757), (3, 0.70710678118654757)]

gensim实际上手使用该目录下data文件夹做为我们的预料进行测试,注:该预料来着人民日报2014年的一部分
其每个文件就是一篇新闻

In [42]:
# -*- coding: utf-8 -*
import jieba,re,os
from gensim.models import word2vec
import logging

对文件进行预处理

In [55]:
# 创建停用词列表
def stopwordslist():
    stopwords = [line.strip() for line in open('stopwords.txt',encoding='UTF-8').readlines()]
    return stopwords
  

In [59]:
def SplitSentence(inputfile,fout,stopwords):      #语料处理：去符号，数字字母 分词 (删除非中文字符)
    fin = open(inputfile,'r',encoding="utf-8")
    for line in fin:
#         line1 = line.strip().decode('utf-8','ignore')
#         line2 = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、★~@#￥%……&*“”➕「」《》（）]+|[０-９0-9a-zA-Z]+", "", line)
        line2 = re.sub("[^\u4e00-\u9fff]+", "", line)
        wordlist = list(jieba.cut(line2))
        outstr = ''
        for word in wordlist:
            if word not in stopwords:
                outstr += word
                outstr += ' '
        fout.write(outstr.strip())
    fout.write('\n')
    fin.close()
def TextLoader(dir):
    fout = open('SplitSentence.txt', 'w',encoding="utf-8")
    stopwords=stopwordslist()
    for root, dirs, files in os.walk(dir):      # 遍历所有文件夹
        for file in files:
            inputfile = os.path.join(root, file)
#             print(inputfile)     #打印文件记录
            SplitSentence(inputfile,fout,stopwords)   #处理该文件
    fout.close()
TextLoader("./data")

训练word2vec

In [61]:
def Training():
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)   #打印日志
    sentences = word2vec.Text8Corpus('SplitSentence.txt')  # 加载语料
    model = word2vec.Word2Vec(sentences)  # 训练skip-gram模型; 默认参数
    model.save('vector.bin')
Training()
#model = word2vec.Word2Vec(sentences, sg=1, size=100, window=5, min_count=5, negative=3, sample=0.001, hs=1, workers=4)
#
# sentences：可以是一个list，对于大语料集，建议使用BrownCorpus,Text8Corpus或·ineSentence构建。
#
# sg：用于设置训练算法，默认为0，对应CBOW算法；sg=1则采用skip-gram算法,对低频词敏感；默认sg=0为CBOW算法。
#
# size是输出词向量的维数，值太小会导致词映射因为冲突而影响结果，值太大则会耗内存并使算法计算变慢，一般值取为几十到几百之间。默认为100
#
# window是句子中当前词与目标词之间的最大距离，3表示在目标词前看3-b个词，后面看b个词（b在0-3之间随机）。
#
# min_count是对词进行过滤，频率小于min-count的单词则会被忽视，默认值为5。
#
# negative和sample可根据训练结果进行微调，sample表示更高频率的词被随机下采样到所设置的阈值，默认值为1e-3。
#
# hs=1表示层级softmax将会被使用，默认hs=0且negative不为0，则负采样将会被选择使用。
#
# workers控制训练的并行，此参数只有在安装了Cpython后才有效，否则只能使用单核。

E:\ProgramFiles\Anaconda3\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "
2019-06-15 16:53:50,959 : INFO : collecting all words and their counts
2019-06-15 16:53:50,961 : WARNING : this function is deprecated, use smart_open.open instead
2019-06-15 16:53:50,967 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-06-15 16:53:51,113 : INFO : collected 32002 word types from a corpus of 177407 raw words and 18 sentences
2019-06-15 16:53:51,116 : INFO : Loading a fresh vocabulary
2019-06-15 16:53:51,300 : INFO : effective_min_count=5 retains 6633 unique words (20% of original 32002, drops 25369)
2019-06-15 16:53:51,302 : INFO : effective_min_count=5 leaves 136915 word corpus (77% of original 177407, drops 40492)
2019-06-15 16:53:51,358 : INFO : deleting the raw counts dictionary of 32002 

2019-06-15 16:59:14,602 : INFO : EPOCH 4 - PROGRESS: at 16.67% examples, 1238 words/s, in_qsize 6, out_qsize 0
2019-06-15 16:59:28,975 : INFO : EPOCH 4 - PROGRESS: at 22.22% examples, 930 words/s, in_qsize 5, out_qsize 0
2019-06-15 16:59:30,261 : INFO : EPOCH 4 - PROGRESS: at 27.78% examples, 1109 words/s, in_qsize 5, out_qsize 0
2019-06-15 16:59:33,328 : INFO : EPOCH 4 - PROGRESS: at 33.33% examples, 1219 words/s, in_qsize 6, out_qsize 0
2019-06-15 16:59:45,549 : INFO : EPOCH 4 - PROGRESS: at 38.89% examples, 1064 words/s, in_qsize 5, out_qsize 0
2019-06-15 16:59:47,923 : INFO : EPOCH 4 - PROGRESS: at 44.44% examples, 1158 words/s, in_qsize 6, out_qsize 0
2019-06-15 16:59:50,699 : INFO : EPOCH 4 - PROGRESS: at 50.00% examples, 1234 words/s, in_qsize 5, out_qsize 0
2019-06-15 17:00:01,596 : INFO : EPOCH 4 - PROGRESS: at 55.56% examples, 1151 words/s, in_qsize 5, out_qsize 0
2019-06-15 17:00:03,865 : INFO : EPOCH 4 - PROGRESS: at 61.11% examples, 1223 words/s, in_qsize 5, out_qsize 0
20

In [62]:
sentences = word2vec.Text8Corpus('SplitSentence.txt')